## Imports

In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

import time

import matplotlib.pyplot as plt
from PIL import Image

from sklearn.model_selection import train_test_split
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings

In [2]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = 128*128
NUM_CLASSES = 8
BATCH_SIZE = 32
DEVICE = 'cuda' # default GPU device
GRAYSCALE = False

# Data Prepare

In [3]:
img_dir = './downloads'
all_folders = os.listdir(img_dir)

In [ ]:
# ## renaming images

# ct = 0
# for ii in all_folders:
#     dd = os.path.join(img_dir, ii)
#     allimg = os.listdir(dd)
#     for idx, filename in enumerate(allimg):
#         os.rename(os.path.join(dd,filename), os.path.join(dd, str(ct+1)+'.jpg'))

        
# ## creating csv
# p0 = os.listdir(os.path.join(img_dir, all_folders[0]))
# p0 = [os.path.join(os.path.join(img_dir, all_folders[0]), im) for im in p0]
# df_tra = pd.DataFrame({'image':p0[:-10], 'label':0})
# df_val = pd.DataFrame({'image':p0[-10:], 'label':0})
# for idx in range(1, len(all_folders)):
#     pp = all_folders[idx]
#     dd = os.path.join(img_dir, pp)
#     intm = os.listdir(dd)
#     ii = [os.path.join(dd, im) for im in intm]
#     #ppp = all_folders[idx+1]
#     #ddd = os.path.join(img_dir, ppp)
#     dft = pd.DataFrame({'image':ii[:-10], 'label':idx})
#     dfv = pd.DataFrame({'image':ii[-10:], 'label':idx})
#     #df2 = pd.DataFrame({'image':os.listdir(ddd), 'class':idx+1})
#     df_tra = pd.concat([df_tra, dft], ignore_index = True)
#     df_val = pd.concat([df_val, dfv], ignore_index = True)

# df_tra.to_csv('train.csv')
# df_val.to_csv('val.csv')

In [6]:
custom_transform = transforms.Compose([transforms.Resize((178, 178)),
                                       transforms.CenterCrop((128, 128)),
                                       #transforms.Grayscale(),                                       
                                       #transforms.Lambda(lambda x: x/255.),
                                       transforms.ToTensor()])
image_datasets=datasets.ImageFolder(r'C:\Users\curti\Desktop\1016-Project\cnn\downloads', custom_transform)

loader = torch.utils.data.DataLoader(image_datasets, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# for idx, (x,y) in enumerate(loader):
#     print(idx, x, y)

### Implementing a Custom DataLoader Class

In [4]:
class SemCogData(Dataset):

    def __init__(self, csv_path,transform=None):
    
        df = pd.read_csv(csv_path, index_col=0)
        self.csv_path = csv_path
        self.img_names = df.image.values
        self.y = df['label'].values
        self.transform = transform

    def __getitem__(self, index):
#         img = Image.open(self.img_names[index])

#         if self.transform is not None:
#             img = self.transform(img)

#         label = self.y[index]
#         return img, label
        try:
            img = Image.open(self.img_names[index])

            if self.transform is not None:
                img = self.transform(img)

            label = self.y[index]
            return img, label
        except:
            print('deleting', self.img_names[index])
            os.remove(self.img_names[index])

    def __len__(self):
        return self.y.shape[0]

p0 = os.listdir(os.path.join(img_dir, all_folders[0]))
p0 = [os.path.join(os.path.join(img_dir, all_folders[0]), im) for im in p0]
df_tra = pd.DataFrame({'image':p0[:-10], 'label':0})
df_val = pd.DataFrame({'image':p0[-10:], 'label':0})
for idx in range(1, len(all_folders)):
    pp = all_folders[idx]
    dd = os.path.join(img_dir, pp)
    intm = os.listdir(dd)
    ii = [os.path.join(dd, im) for im in intm]
    #ppp = all_folders[idx+1]
    #ddd = os.path.join(img_dir, ppp)
    dft = pd.DataFrame({'image':ii[:-10], 'label':idx})
    dfv = pd.DataFrame({'image':ii[-10:], 'label':idx})
    #df2 = pd.DataFrame({'image':os.listdir(ddd), 'class':idx+1})
    df_tra = pd.concat([df_tra, dft], ignore_index = True)
    df_val = pd.concat([df_val, dfv], ignore_index = True)

df_tra.to_csv('train.csv')
df_val.to_csv('val.csv')
# Note that transforms.ToTensor()
# already divides pixels by 255. internally

custom_transform = transforms.Compose([transforms.CenterCrop((178, 178)),
                                       transforms.Resize((128, 128)),
                                       #transforms.Grayscale(),                                       
                                       #transforms.Lambda(lambda x: x/255.),
                                       transforms.ToTensor()])

train_dataset = SemCogData(csv_path='train.csv',
                              transform=custom_transform)

valid_dataset = SemCogData(csv_path='val.csv',
                              transform=custom_transform)


train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4, drop_last=True)

valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4, drop_last=True)
for idx in range(len(train_dataset)):
    a = train_dataset[idx]
    
for idx in range(len(valid_dataset)):
    a = valid_dataset[idx]

C:\Users\curti\anaconda3\lib\site-packages\PIL\Image.py:2966: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


deleting ./downloads\0pine\110.nearly-natural-artificial-trees-5694-rd-64_600.jpg.webp
deleting ./downloads\0pine\13.scotchpinetree__44838-200x200__17373__77155.1513076159.jpg.webp
deleting ./downloads\0pine\153.word-image-2.jpeg.webp
deleting ./downloads\0pine\16.korean_pine__60240-400x400__54314__60590.1512125788.jpg.webp
deleting ./downloads\0pine\160.01_europe_oldest_tree_img_20160727_131319.jpg.webp
deleting ./downloads\0pine\188.009259607039.jpg.webp
deleting ./downloads\0pine\216.23__09205__83989.1512125387.jpg.webp
deleting ./downloads\0pine\220.packpine05screenshot-1920x1080-26db2cf24a4e4cffbc58c3d628bb342c.jpg.webp
deleting ./downloads\0pine\268.01_italus3.jpg.webp
deleting ./downloads\0pine\350.coulter-pine-with-cones_bryant-baker-1024x683-1.jpg.webp
deleting ./downloads\0pine\364.evergreen-nursery-shade-trees-pinschaqt-64_400.jpg.webp
deleting ./downloads\0pine\396.20180910-dsc09970__37335.1574709175.jpg.webp
deleting ./downloads\0pine\52.packpine01screenshot-1920x1080-d37b

In [7]:
torch.manual_seed(0)

for epoch in range(2):

    for batch_idx, (x, y) in enumerate(loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(DEVICE)
        y = y.to(DEVICE)
        time.sleep(1)
        break

Epoch: 1 | Batch index: 0 | Batch size: 32
Epoch: 2 | Batch index: 0 | Batch size: 32


## Model

The following code cell that implements the ResNet-34 architecture is a derivative of the code provided at https://pytorch.org/docs/0.4.0/_modules/torchvision/models/resnet.html.

In [8]:
##########################
### MODEL
##########################


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out




class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes, grayscale):
        self.inplanes = 64
        if grayscale:
            in_dim = 1
        else:
            in_dim = 3
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1, padding=2)
        self.fc = nn.Linear(2048 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n)**.5)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.fc(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas



def resnet101(num_classes, grayscale):
    """Constructs a ResNet-101 model."""
    model = ResNet(block=Bottleneck, 
                   layers=[3, 4, 23, 3],
                   num_classes=NUM_CLASSES,
                   grayscale=grayscale)
    return model

In [9]:
torch.manual_seed(RANDOM_SEED)

##########################
### COST AND OPTIMIZER
##########################

model = resnet101(NUM_CLASSES, GRAYSCALE)
model.to(DEVICE)

cost_fn = torch.nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)  

## Training

In [10]:
train_loader = loader
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = cost_fn(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
              #,| Valid: %.3f%% compute_accuracy(model, valid_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 0000/0097 | Cost: 2.0943
Epoch: 001/010 | Batch 0050/0097 | Cost: 1.7822
Epoch: 001/010 | Train: 41.386%
Time elapsed: 0.80 min
Epoch: 002/010 | Batch 0000/0097 | Cost: 1.6077
Epoch: 002/010 | Batch 0050/0097 | Cost: 1.8864
Epoch: 002/010 | Train: 52.364%
Time elapsed: 1.59 min
Epoch: 003/010 | Batch 0000/0097 | Cost: 1.2732
Epoch: 003/010 | Batch 0050/0097 | Cost: 1.0757
Epoch: 003/010 | Train: 39.864%
Time elapsed: 2.38 min
Epoch: 004/010 | Batch 0000/0097 | Cost: 1.1459
Epoch: 004/010 | Batch 0050/0097 | Cost: 1.2331
Epoch: 004/010 | Train: 55.861%
Time elapsed: 3.14 min
Epoch: 005/010 | Batch 0000/0097 | Cost: 1.4254
Epoch: 005/010 | Batch 0050/0097 | Cost: 1.3759
Epoch: 005/010 | Train: 61.626%
Time elapsed: 3.91 min
Epoch: 006/010 | Batch 0000/0097 | Cost: 1.4565
Epoch: 006/010 | Batch 0050/0097 | Cost: 0.8224
Epoch: 006/010 | Train: 56.412%
Time elapsed: 4.71 min
Epoch: 007/010 | Batch 0000/0097 | Cost: 1.2244
Epoch: 007/010 | Batch 0050/0097 | Cost: 1.498

## Evaluation

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

In [ ]:
for batch_idx, (features, targets) in enumerate(loader):

    features = features
    targets = targets
    break
    
plt.imshow(np.transpose(features[0], (1, 2, 0)))

In [ ]:
plt.imshow(np.transpose(features[5], (1, 2, 0)))

In [ ]:
targets

In [ ]:
model.eval()
logits, probas = model(features.to(DEVICE)[0, None])
print('Probability Female %.2f%%' % (probas[0][0]*100))

In [ ]:
%watermark -iv